In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None) #to supress column truncation
# pd.set_option('display.max_rows', None) 

# Importing data and reading to dfs to combine into single df

### Reading the `311_service_data` csv to a df.

In [2]:
service_data = pd.read_csv('311_service_data.csv')
service_data.head(2)

,Category,CASEID,OPENEDDATETIME,SLA_Date,CLOSEDDATETIME,Late (Yes/No),Dept,REASONNAME,TYPENAME,CaseStatus,SourceID,OBJECTDESC,Council District,XCOORD,YCOORD,Report Starting Date,Report Ending Date
0,Graffiti,1010444245,8/15/2012,8/30/2012,NaN,YES,Code Enforcement Services,Graffiti,Graffiti Public Property,Open,Web Portal,"600 NOGALITOS ST, San Antonio, 78204",5,2125683.0,13695548.0,1/15/2021,1/15/2022
1,Property Maintenance,1010888252,6/6/2013,8/9/2013,NaN,YES,Code Enforcement Services,Code Enforcement (IntExp),Alley-Way Maintenance,Open,Web Portal,"6043 CASTLE QUEEN, San Antonio, 78218",2,2169702.0,13725769.0,1/15/2021,1/15/2022


In [3]:
service_data.shape

(521590, 17)

### Reading the `med_incomebyzip` csv to a df.

In [4]:
med_incomebyzip = pd.read_csv('med_incomebyzip.csv').drop(columns = ['#'])
med_incomebyzip.head(2)

,Zip Code,Location,Population,avg_household_income,Lat,Long
0,78261,"29.705479, -98.402849","1,119","$92,154.00",29.705479,-98.402849
1,78258,"29.647111, -98.500768","17,355","$91,509.00",29.647111,-98.500768


In [5]:
med_incomebyzip.shape

(60, 6)

### I could do an inner join on zip code and combine the dataframes. 
First, I need to modify the `OBJECTDESCR` address values in the `service_data` df to only show the zip codes. I can do this by calling the last 5 (-5) values.
- Some addresses do not have a zip code. The inner join will eliminate those values from the joined df. Given additional time, I will come back and look up the zip codes for the address that do not have them.

In [6]:
# creating a for loop that will loop through each address value and give me the last 5 character in the string 
zips = []

for address in service_data.OBJECTDESC:
    zips.append(address[-5:])
    
# creating a new zip column for the df with those values
service_data['zip'] = zips
    
# dropping the OBJECTDESC address column from the df
service_data.drop(columns = ['OBJECTDESC'], inplace = True)
service_data.head(2)

,Category,CASEID,OPENEDDATETIME,SLA_Date,CLOSEDDATETIME,Late (Yes/No),Dept,REASONNAME,TYPENAME,CaseStatus,SourceID,Council District,XCOORD,YCOORD,Report Starting Date,Report Ending Date,zip
0,Graffiti,1010444245,8/15/2012,8/30/2012,NaN,YES,Code Enforcement Services,Graffiti,Graffiti Public Property,Open,Web Portal,5,2125683.0,13695548.0,1/15/2021,1/15/2022,78204
1,Property Maintenance,1010888252,6/6/2013,8/9/2013,NaN,YES,Code Enforcement Services,Code Enforcement (IntExp),Alley-Way Maintenance,Open,Web Portal,2,2169702.0,13725769.0,1/15/2021,1/15/2022,78218


In [7]:
# the following line of code will allow me to see all of the rows so I can look at the nonnumeric values


# nonzips = []
# for value in service_data.zip:
#     if value[0] != '7':
#         nonzips.append(value)

>### I am getting an error here: it apppears that the columns I am trying to merge on do not have the same dtypes

In [8]:
service_data.merge(med_incomebyzip, how = 'inner',
                  left_on = 'zip', right_on = 'Zip Code')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
service_data.zip.dtype

In [ ]:
med_incomebyzip['Zip Code'].dtype

#### The first dataframe's zip codes are objects and the second's are integers
>#### Since these will need to be objects I will convert the second dataframes zip codes to object values

In [ ]:
# changing series dtype from int to object
med_incomebyzip['Zip Code'] = med_incomebyzip['Zip Code'].astype('O')

In [ ]:
med_incomebyzip['Zip Code'].dtype

In [ ]:
type(med_incomebyzip['Zip Code'][0])

In [ ]:
Zip_Code = []

for value in med_incomebyzip['Zip Code']:
    Zip_Code.append(str(value))
    
Zip_Code[:5]

In [ ]:
# replacing int Zip Code column with strings
med_incomebyzip['Zip Code'] = Zip_Code
type(med_incomebyzip['Zip Code'][0])

### Going to try the merge again...

In [ ]:
df = service_data.merge(med_incomebyzip, how = 'inner',
                  left_on = 'zip', right_on = 'Zip Code').drop(columns = ['Zip Code'])
df.head(2)

### And we have a merged df!

# General Info About the Data

In [ ]:
print(f'There are {df.shape[0]} observations and {df.shape[1]} columns in the data.')

In [ ]:
df.head(2)

In [ ]:
df['Report Starting Date'].value_counts()

In [ ]:
df['Report Ending Date'].value_counts()

### I will go ahead and start removing some of the columns I know I will not be using
I will also lowercase the columns names here as well
- `TYPENAME` pretty similar to `REASONNAME` and accepts multiple values
- `XCOORD` and `YCOORD` unsure of coordinate system used? Does not appear to coorespond with lat/long, dropping for now
- `Report Starting Date` and `Report Ending Date` since they have only x1 value/are the same for each report

In [ ]:
df.columns.str.replace(' ', '').str.lower()

In [ ]:
cols_to_drop = ['TYPENAME', 'XCOORD', 'YCOORD', 'Report Starting Date', 'Report Ending Date']

df.drop(columns = cols_to_drop, inplace = True)

df.columns = df.columns.str.replace(' ', '').str.lower()

df.head(2)

### And while I am it, I will do a little more column name cleanup and make them more readable based on the data dictionary description

In [ ]:
cols_to_rename = {'caseid':'case_ref_num', 
'openeddatetime':'case_open', 
'sla_date':'sla_due', 
'closeddatetime':'case_close', 
'late(yes/no)':'sla_late', 
'reasonname':'dept_div',  
'casestatus':'case_status', 
'councildistrict':'council_distr', 
'avg_household_income':'avg_inc'}

df.rename(columns = cols_to_rename, inplace = True)
df.head(2)

### Looking at dtypes for each column...

In [ ]:
# creating empty lists to be appended in for loop
obj_list = []
num_list = []
bool_list = []

# for loop to append df columns to corresponding lists
for col in df.columns:
    if df[col].dtype == 'O':
        obj_list.append(col)
    elif df[col].dtype == 'bool':
        bool_list.append(col)
    else:
        num_list.append(col)
        
print(f'Object List:\n{obj_list}\n\nNumeric List:\n{num_list}\n\nBool List:\n{bool_list}')

### Columns I will need to update `dtype` for:
- `case_ref_num` from numeric to object
- `case_open` from object to datetype 
- `sla_due` from object to datetype 
- `case_close` from object to datetype 
- `council_distr` from number to object
- `population` from object to integer
    - remove `,` 
- `avg_inc`
    - first need to remove `$` and `,`
    - from object to float
    
    
 Also, `location` column is redudant, will drop that column

In [ ]:
# to object
df[['case_ref_num', 'council_distr']] = df[['case_ref_num', 'council_distr']].astype('O')

# to datetime
pd.to_datetime(df[['case_open', 'sla_due', 'case_close']], format = '%Y:%m:%d')

### I think I am getting this error because of null values
I am going to drop null values and then come back to column dtype cleanup

In [ ]:
df[['case_open', 'sla_due', 'case_close']].isna().sum()

### Let's take a look at null values....

In [ ]:
nulls = pd.concat([round(df.isna().sum().sort_values(ascending = False) / df.shape[0] * 100, 2), \
                     df.isna().sum().sort_values(ascending = False)], axis = 1).head(4)
nulls

#### The null values are a very small percentage of the total data and I don't think I will lose a signficant amount info by dropping them

In [ ]:
df.dropna(inplace = True)

In [ ]:
# verying there are no null values
df.isna().sum().sum()

#### Going to try the `datetime` dtype conversion again...

In [ ]:
# to datetime
pd.to_datetime(df[['case_open', 'sla_due', 'case_close']], format = '%Y:%m:%d')

#### Still getting an error, but code works without `format` and doing each column individually. 

In [ ]:
# to datetime
df.sla_due = pd.to_datetime(df.sla_due)
df.case_open = pd.to_datetime(df.case_open)
df.case_close = pd.to_datetime(df.case_close)

# population from object to integer
df.population = df.population.str.replace(',', '').astype(int)

# to float
df.avg_inc = df.avg_inc.str.replace('$', '').str.replace(',', '').astype(float)

In [ ]:
# dropping location column
df.drop(columns =  ['location'], inplace = True)

### For the last cleanup step I am feature engineering a new column `days_to_close`  
- This will be how many days it took for the case to close
- Derived from `case_closed` - `case_open`

In [ ]:
# converting datetime to string
# removing non-numeric endings
# changing dtype to integer
df['days_to_close'] = (df.case_close - df.case_open).astype('str').str.replace(' days', '').astype(int)

## Now to look at the columns and dtypes to verify cleanup and ready for explore...

In [ ]:
df.head(2)

### I want to reset the `index` to get a continuous value. After dropping some data the numbers skip around...

In [ ]:
# df.reset_index(inplace = True)

df = df.rename(columns = {'index':'index_'}).drop(columns = ['level_0', 'index_'])

In [ ]:
df.head(2)

In [ ]:
# creating empty lists to be appended in for loop
obj_list = []
int_list = []
float_list = []
bool_list = []
date_list = []

# for loop to append df columns to corresponding lists
for col in df.columns:
    if df[col].nunique() == 2:
        bool_list.append(col)
        
    elif df[col].dtype == 'O':
        obj_list.append(col)
        
    elif df[col].dtype == 'int':
        int_list.append(col)

    elif df[col].dtype == 'float':
        float_list.append(col)
    
    else:
        date_list.append(col)
        
print(f'Object List:\n{obj_list}\n\nInteger List:\n{int_list}\n\nBool List:\n{bool_list}\n\nFloat List:\n{float_list}\n\nDates List:\n{date_list}')

### I can also drop the `case_status` column since I removed all the null values for `case_close` 
- the only cases I will be looking at are the ones who have been closed as of the data report date

In [ ]:
df.drop(columns = ['case_close'], inplace = True)

### also dropping the `case_ref_num` column since it is unique for each value (can use index as counter)

In [ ]:
df.drop(columns = ['case_ref_num'], inplace = True)

In [ ]:
df.head(2)

## Data Dictionary

Target | Dtype | Description
:--- | :--- | :---
`days_to_close` | datetime | the total time in days it took to close the case; feature engineered from `close_date` - `open_date`


Variable | Dtype |  Description
:--- | :--- | :---
`category` | object | top level 311 service request category
`case_open` | datetime | the date a case was submitted
`sla_due` | dtype | each service request `category` has a due date assigned to the request, based on the dept division `dept_div`
`dept` | object | the City deaprtment to whom the case is assigned
`dept_div` | object | the department division within the City deaprtment to whom the case is assigned
`council_distr` | object | The Council District number from where the issue was reported
`zip` | object | the zip code for the reported case/service requested
`population` | int | the population for the zip code for the reported case/service requested
`avg_inc` | float | the avergae income for the zip code for the reported case/service requested
`lat` | float | the latitude coordinate for the zip code for the reported case/service requested
`long` | float | the longitude coordinate for the zip code for the reported case/service requested


In [ ]:
df.info()

In [ ]:
df.days_to_close.value_counts()

### The last step in wrangling the data is splitting...
For this pass I am not going to stratify
>On the next pass I will come back and create a column with quartile bins for my target `days_to_close` and stratify on that variable column

In [ ]:
from sklearn.model_selection import train_test_split

# creating test dataset
train_validate, test = train_test_split(df, test_size=.2, random_state=12)

# creating the train and test datasets
train, validate = train_test_split(train_validate, test_size=.3, random_state=12)

# verifying the split
print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

In [ ]:
train.head()

## df is ready for exploration!